In [ ]:
import bs4
import requests
import pandas as pd
import time
import random
import math
import sqlite3
import numpy as np
from math import pi
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, output_file, show    
from google.colab import drive

In [ ]:
class coinscrapper(object):
  today = None
  kucoin_list = []
  technical_data = []
  fundamental_data = []
  rsi_data = []
  todays_RSI = []
  marketcaps = []
  volumes = []
  plots = []
  log = ""
  test= None

  def __init__(self, today, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.today = today
    self.log.join("Loading client\n")
    self.connect_googleDrive()
  
  def connect_googleDrive(self):
    drive.mount('drive', force_remount=True)
    self.log.join("Connecting to Google Drive\n")

  def get_kc_list(self):
    today_ = self.today
    try:
      from selenium import webdriver
      options = webdriver.ChromeOptions()
      options.add_argument('user-agent= Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36')
      options.add_argument('-headless')
      options.add_argument('-no-sandbox')
      options.add_argument('-disable-dev-shm-usage')
      urls = []
      with webdriver.Chrome('chromedriver',options=options) as wd:
          url = "https://coinmarketcap.com/exchanges/kucoin/"
          wd.get(url)
          def scroll_to_end(wd2, scroll_point = 'document.body.scrollHeight'):  
            for i in range(1,1000):
                print(i)
                wd2.execute_script(f"window.scrollTo(0, {i*scroll_point});")
                time.sleep(7) 
                try:
                    load_more_button = wd2.find_element_by_xpath('//div[@class="cmc-markets-listing__load-more")')
                    if load_more_button:
                        load_more_button.click()
                        time.sleep(3) 
                except:
                    break
          scroll_to_end(wd)
          elem = wd.find_elements_by_xpath("//tr//div//a[@class='cmc-link']")
          res = [j.get_attribute('href') for j in elem]   
          print(res)
          for k in res:
            if 'trade' not in k:
              urls.append(k)
      historic_urls = [l + 'historical-data/'+ f'?start=20200501&end={today_}' for l in set(urls)]
      print(historic_urls)
      self.kucoin_list = historic_urls
      self.log.join("Successfully pulled link Kucoin assest from CMC\n")
      return historic_urls

    except:
      print('Do you want to install the required packages?\ny/n' )
      x = input()
      if x == 'y':
        print ('okay. We will install the required packages.')
        !apt-get update 
        !apt install chromium-chromedriver
        !cp /usr/lib/chromium-browser/chromedriver /usr/bin
        !pip install selenium
        !pip install progress
      else:
        print ('Sorry you need these packages to continue.\n Try again? y/n')
        y = input()
        if y == "y":
          self.get_kc_list()
          print('Success!')
        else:
          pass
      self.log.join("Failed to get link Kucoin assest from CMC\n")

  def get_data(self):
    if len(self.kucoin_list) < 1:
      self.get_kc_list()
    series = self.kucoin_list
    time.sleep(random.uniform(7, 10))
    start = time.time()
    for idx, coins in enumerate(series):
      loop = time.time()
      print(idx+1,"out of ", len(series))
      try:
        data = pd.read_html(coins)
        self.technical_data.append(data[2])
        self.fundamental_data.append(data[3])
        time.sleep(random.uniform(7, 10))
        elapsed = loop - start
        print('Success!', elapsed, 'seconds...')
        self.log.join(f'{round(elapsed/60,2)} minutes elapsed...\n')
      except:
        self.technical_data.append("Error")
        self.fundamental_data.append("Error")
        time.sleep(random.uniform(7, 10))
        elapsed = loop - start
        print("Failed.", elapsed, 'seconds...')
        self.log.join(f'Ooopps.... Somebody hating. \nWe missing the one for {coins}, \n or CMC just did not have the data')
        self.log.join(f'{round(elapsed/60,2)} minutes elapsed...\n')

  def computeRSI (self, data, time_window):
    diff = data.diff(1).dropna()  
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    self.log.join("Computing RSI\n")
    return rsi

  def preprocess(self, df):
    data = df.copy()
    data['date_'] = pd.to_datetime(data['Date'])
    data.index = pd.DatetimeIndex(data['date_'])
    data.sort_index(ascending=True, inplace=True)
    self.log.join("Prepocessing data\n")
    return data

  def get_rsi_data(self, data):
    i = data.copy
    x = self.preprocess(i)
    x['RSI'] = computeRSI(x['Close**'], 14)
    r = list(x['RSI'])[-1]
    self.rsi_data.append(x)
    self.todays_RSI.append(r)
    self.log.join("Collecing RSI data\n")
    return x

  def plot_rsi_data(self, data, title):
    title = title
    test = data 
    r = list(test['RSI'])[-1]
    print(f'Plotting RSI ({r}) for: ', f'{title}')
    plt.figure(figsize=(40,9))
    plt.plot(test.index[-14:],test.RSI[-14:])
    plt.title(f'{title}'+"-Current-RSI-"+str(r))
    plt.hlines(80,xmin=test.index[-14:].min(),  xmax=test.index[-14:].max(),colors='k', label='80')
    plt.hlines(70,xmin=test.index[-14:].min(),  xmax=test.index[-14:].max(),colors='r', label='70', linestyles='dashdot')
    plt.hlines(30,xmin=test.index[-14:].min(),  xmax=test.index[-14:].max(),colors='g', label='30', linestyles='dashdot')
    plt.hlines(20,xmin=test.index[-14:].min(),  xmax=test.index[-14:].max(),colors='b', label='20')
    plt.xlabel('Date')
    plt.ylabel('RSI_14')
    plt.legend()
    plt.show();
    try:
      plt.savefig(f'drive/My Drive/CoinScraper/charts/{j}.png', dpi=72)
      self.plots.append(plt)
      self.log.join(f"Plotting {j} data\n")
    except:
      self.connect_googleDrive()
      plt.savefig(f'drive/My Drive/CoinScraper/charts/{j}.png', dpi=72)
      self.plots.append(plt)
      self.log.join(f"Plotting {j} data\n")

  def summary(self):
    if len(self.technical_data) < 1:
      self.get_data()
    datasets = self.technical_data
    titles = self.kucoin_list
    for i, j in zip(datasets, titles):
      self.get_rsi_data(i)
      self.plot_rsi_data(i,j)
    sum_table = pd.DataFrame()
    if len(titles) == len(self.todays_RSI):
      sum_table['URL'] = titles
      sum_table['RSI'] = self.todays_RSI
      return sum_table.sort_values(by='RSI', axis=1)


  def save_candlesticks(self):
    if len(self.technical_data) < 1:
      self.get_data()
    if len(self.plots) < 1:
      self.plot_rsi_data()
    for i, j in zip(self.technical_data, self.kucoin_list):
      try:
        df = i
        inc = df['Close**'] > df['Open*']
        dec = df['Open*'] > df['Close**']
        w = 12*60*60*1000 # half day in ms
        TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
        p = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, title = "{j} Price Chart")
        p.xaxis.major_label_orientation = pi/4
        p.grid.grid_line_alpha=0.3
        p.segment(df.index, df['High'], df.index, df['Low'], color="black")
        p.vbar(df.index[inc], w, df['Open*'][inc], df['Close**'][inc], fill_color="#D5E1DD", line_color="black")
        p.vbar(df.index[dec], w, df['Open*'][dec], df['Close**'][dec], fill_color="#F2583E", line_color="black")
        output_file(f"drive/My Drive/CoinScraper/charts/{j}/{j}.html", title="candlestick.py example")
        self.log.join(f'Plotting {j} candlesticks\n')
        show(p)  # open a browser
      except:
        self.log.join(f'Error plotting candlesticks for {j}\n')

  def pull_fundamentals(self):
    if len(self.fundamental_data) < 1:
      self.get_data()
    if len(self.todays_RSI) < 1:
      self.get_rsi_data()
    fund = self.fundamental_data
    for i in fund:
      try:
        marketcap = i.set_index(0).stack()[3]
        volume = i.set_index(0).stack()[4]
        self.marketcaps.append(marketcap)
        self.volumes.append(volume)
        self.log.join('Pulling fundamentals\n')
      except:
        self.marketcaps.append('Error')
        self.volumes.append('Error')
        self.log.join('Error pulling fundamentals\n')
      
  def load_images():
    pass


  def summarize(self):
    if len(self.fundamental_data) < 1:
      self.pull_fundamentals()
    if len(self.todays_RSI) < 1:
      self.get_rsi_data()
    results = pd.DataFrame()
    results['Date'] = self.today
    results['Asset'] = self.kucoin_list
    results['MarketCap'] = self.marketcaps
    results['Volume'] = self.volumes
    results['RSI'] = self.todays_RSI
    results['Price'] = self.todays_price
    # results.sort_values(by='RSI', axis=1, inplace=True)
    # results['Plots']
    try:
      results.to_csv('drive/My Drive/CoinScraper/summary.csv')
      self.log.join('Saving Data\n')
    except:
      self.connect_googleDrive()
      results.to_csv('drive/My Drive/CoinScraper/summary.csv')
    return results.to_html()

# Connecting to Client

In [ ]:
client = coinscrapper('20200818')

# Getting list of Kucoin assests

In [ ]:
start = time.time()
client.get_kc_list()
end = time.time()
print(end-start, 'seconds')

# Requesting Data from CMC

In [ ]:
import random
client.get_data()

# Computing RSI data

In [ ]:
client.summary()

Success! 189.53978037834167 seconds...
23 out of  203


# Plot RSI Data

In [ ]:
cliet.plot_rsi_data()

# Summary

In [ ]:
testing.save_candlesticks()

In [ ]:
testing.log

In [ ]:
testing.fundamental_data

In [ ]:
testing.technical_data